# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [3]:
Pkg.activate("..") # activate the parent environment
using BootstrapVirasoro
n_threads = Threads.nthreads()
if n_threads == 1
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`")
end

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [ ]:
setprecision(BigFloat, 35, base=10)
c = CC(β=1/(big"0.8" + big"0.1"*im))
Δmax = 40.

function LoopFields(model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(model, Δmax) = Spectrum(LoopFields(model), Δmax, interchiral=true);

function solve_bootstrap(indices, Δmax; signature = Dict(:s => 0, :t => 0, :u => 0), diags = Dict())
    fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(fields..., Δmax=Δmax)
    SOn = LoopSpectrum(:On, Δmax)

    Schan_On = ChannelSpectra(
        co, SOn,
        signature=signature
    )

    for (chan, VP) in diags
        add!(Schan_On, VP)
    end

    b = BootstrapSystem(Schan_On)
    compute_linear_system!(b)
    solve!(b)
    return b
end

solve_bootstrap (generic function with 1 method)

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

We expect a three-dimensional space of solutions. We can distinguish between them by choosing an appropriate signature.

In [7]:
indices = ((1 // 2, 0), (1 // 2, 0), (1, 0), (1, 0))
system1 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 1, :t => 1//2, :u => 3//2))
system2 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 0, :t => 3, :u => 3//2))
system3 = solve_bootstrap(indices, Δmax, signature=Dict(:s => 1, :t => 3//2, :u => 1//2));

┌ Info: system size: (158, 152)
└ @ BootstrapVirasoro /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/bootstrap_equations.jl:193
┌ Info: system size: (150, 143)
└ @ BootstrapVirasoro /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/bootstrap_equations.jl:193
┌ Info: system size: (158, 152)
└ @ BootstrapVirasoro /Users/paulroux/Documents/Recherche/projet_these/code/BootstrapVirasoro/src/bootstrap_equations.jl:193


In [8]:
println(system.consts)
println(system2.consts)
println(system3.consts)

Channel s
Fields         | Structure constants          | Relative errors            
--------------------------------------------------------------------
V_{(1, 0)}     |  1.00000e+00 + 0.00000e+00im | 0.00000e+00 + 0.00000e+00im
V_{(1, 1)}     | -1.49074e-03 - 1.81475e-03im |  2.02042e-24 + 0.00000e+00im
V_{(2, 0)}     |  2.73704e-03 + 2.00265e-02im |  1.76788e-24 + 0.00000e+00im
V_{(2, 1//2)}  | -3.06245e-27 + 6.43662e-27im |  7.22968e-01 + 0.00000e+00im
V_{(2, 1)}     |  1.40779e-07 - 1.16718e-07im |  2.40049e-17 + 0.00000e+00im
V_{(2, 3//2)}  |  2.39155e-03 + 2.55139e-02im |  2.27496e-24 + 0.00000e+00im
V_{(3, 0)}     | -7.07647e-07 - 9.43683e-07im |  2.37925e-17 + 0.00000e+00im
V_{(3, 1//3)}  | -6.00257e-07 + 2.35486e-07im |  1.55943e-17 + 0.00000e+00im
V_{(3, 2//3)}  |  3.77202e-24 + 4.43535e-24im |  4.45249e-01 + 0.00000e+00im
V_{(3, 1)}     | -6.37283e-13 - 2.34754e-14im |  1.82008e-05 + 0.00000e+00im
V_{(3, 4//3)}  |  9.06899e-08 + 2.68347e-08im |  2.74859e-17 + 0.00000e+00im